In [198]:
from tqdm import tqdm
from pulp import *

with open('input.txt') as file:
    data = file.read().splitlines()

In [199]:
lp = []

for line in data:
    rob = line.split('.')

    lp.append([
        [int(rob[0].split(' ')[-2]), 0, 0, 0],
        [int(rob[1].split(' ')[-2]), 0, 0, 0],
        [int(rob[2].split(' ')[-5]), int(rob[2].split(' ')[-2]), 0, 0],
        [int(rob[3].split(' ')[-5]), 0, int(rob[3].split(' ')[-2]), 0],
    ])

In [244]:
def solve(blueprint, minutes):

  ROBOTS = range(4)
  MINUTES = range(minutes)

  prob = LpProblem("aoc", LpMaximize)

  choices = LpVariable.dicts("Choice", (ROBOTS, MINUTES), cat="Binary")
  stones_produced = LpVariable.dicts("stones_produced", (ROBOTS, MINUTES), cat="Integer")
  stones_inventory = LpVariable.dicts("stones_inventory", (ROBOTS, MINUTES), cat="Integer")
  cost_per_round = LpVariable.dicts("cost_per_round", (ROBOTS, MINUTES), cat="Integer")
  cumulative_costs = LpVariable.dicts("cum_costs", (ROBOTS, MINUTES), cat="Integer")

  # Set target
  prob += lpSum([stones_inventory[len(ROBOTS) - 1][minutes-1]])

  # Add constrains
  for m in MINUTES:

    prob += lpSum([choices[r][m] for r in ROBOTS]) >= 0
    prob += lpSum([choices[r][m] for r in ROBOTS]) <= 1

    for r in ROBOTS:

      if m >= minutes - 2:
        prob += cost_per_round[r][m] == 0
        prob += cost_per_round[r][m] == 0
      else:
        prob += sum([choices[r_][m + 2] * blueprint[r_][r] for r_ in ROBOTS]) == cost_per_round[r][m]
      
      
      prob += (lpSum([cost_per_round[r][m_] for m_ in range(m + 1)])) == cumulative_costs[r][m]
      prob += (lpSum([choices[r][m_] for m_ in range(m + 1)])) == stones_produced[r][m]

      prob += lpSum([
                stones_produced[r][m_] 
                for m_ in range(m + 1)
                ] + [-cumulative_costs[r][m]]
              ) == stones_inventory[r][m]

      prob += stones_inventory[r][m] >= 0


  # Set starting conditions
  prob += choices[0][0] == 1
  prob += lpSum([choices[r][1] for r in ROBOTS]) == 0
      
  prob.solve(PULP_CBC_CMD(msg=0))
  return value(stones_inventory[len(ROBOTS) - 1][minutes - 1])

In [245]:
part_1 = 0

for idx, blueprint in enumerate(lp):
  part_1 += (idx + 1) * solve(blueprint, 24)

part_1

1356.0

In [243]:
part_2 = 1

for idx, blueprint in enumerate(lp[:3]):
  part_2 *= solve(blueprint, 32)

part_2

27720.0